In [38]:
# autoreload
%load_ext autoreload
%autoreload 2





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Index(['date', 'S0', 'rf'], dtype='object')


In [63]:
# autoreload
%load_ext autoreload
%autoreload 2

from src.data import read_fund_portfolio_options
fund_portfolio_options = read_fund_portfolio_options(fund='aram')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
fund_portfolio_options

,stock_symbol,holding,option_symbol,maturity_date,strike,type
0,خودرو,38372289,ضخود6015,1402-06-08,3000,call
1,خودرو,38372289,طخود6015,1402-06-08,3000,put
2,وتجارت,27555013,ضجار6010,1402-06-14,2942,call
3,وتجارت,27555013,طجار6010,1402-06-14,2942,put
4,وبصادر,30935774,ضصاد7008,1402-07-12,2797,call
5,وبصادر,30935774,طصاد7008,1402-07-12,2797,put
6,وبملت,57146213,ضملت7003,1402-07-26,4370,call
7,وبملت,57146213,طملت7003,1402-07-26,4370,put
8,شپنا,32895084,ضشنا6009,1402-06-01,14000,call
9,شپنا,32895084,طشنا6009,1402-06-01,14000,put


In [65]:
from src.data import fetch_stoch_history
from src.data import add_rf, add_T, add_std
from src.options import Strategy

In [67]:
fund_portfolio_options['delta'] = None
# fund_portfolio_options['gamma'] = None
fund_portfolio_options['vega'] = None
fund_portfolio_options['theta'] = None
# fund_portfolio_options['rho'] = None

fund_portfolio_options['delta_neutral'] = None
# fund_portfolio_options['vega_neutral'] = None
# fund_portfolio_options['theta_neutral'] = None

for option in fund_portfolio_options.iterrows():

    symbol = option[1]['stock_symbol']
    stock_price_df = fetch_stoch_history(symbol)
    stock_price_df = add_rf(stock_price_df)
    stock_price_df = add_std(stock_price_df)
    date, S0, rf, std = stock_price_df.iloc[-1]

    maturity_date = option[1]['maturity_date']
    strike = option[1]['strike']
    is_call = option[1]['type'] == 'call'

    delta, gamma, vega, theta, rho = (
        Strategy.add_greeks(K=strike, mat_date=maturity_date, is_call=is_call, today=str(date), S0=S0, rf=rf, sigma=std)
    )
    fund_portfolio_options.loc[option[0], 'delta'] = delta
    # fund_portfolio_options.loc[option[0], 'gamma'] = gamma
    fund_portfolio_options.loc[option[0], 'vega'] = vega
    fund_portfolio_options.loc[option[0], 'theta'] = theta
    # fund_portfolio_options.loc[option[0], 'rho'] = rho

    fund_portfolio_options.loc[option[0], 'delta_neutral'] = Strategy.delta_neutral_strategy(delta, option[1]['holding'])
    # fund_portfolio_options.loc[option[0], 'vega_neutral'] = Strategy.vega_neutral_strategy(vega, option[1]['holding'])
    # fund_portfolio_options.loc[option[0], 'theta_neutral'] = Strategy.theta_neutral_strategy(theta, option[1]['holding'])

In [68]:
fund_portfolio_options

,stock_symbol,holding,option_symbol,maturity_date,strike,type,delta,vega,theta,delta_neutral
0,خودرو,38372289,ضخود6015,1402-06-08,3000,call,0.774138,383.160371,-1020.14514,49567.754081
1,خودرو,38372289,طخود6015,1402-06-08,3000,put,-0.774138,383.160371,-265.645197,-49567.754081
2,وتجارت,27555013,ضجار6010,1402-06-14,2942,call,0.32102,374.262428,-714.299941,85835.772023
3,وتجارت,27555013,طجار6010,1402-06-14,2942,put,-0.32102,374.262428,-7.114425,-85835.772023
4,وبصادر,30935774,ضصاد7008,1402-07-12,2797,call,0.342169,432.727976,-552.798466,90410.875876
5,وبصادر,30935774,طصاد7008,1402-07-12,2797,put,-0.342169,432.727976,106.246961,-90410.875876
6,وبملت,57146213,ضملت7003,1402-07-26,4370,call,0.758921,813.778706,-1474.116426,75299.247468
7,وبملت,57146213,طملت7003,1402-07-26,4370,put,-0.758921,813.778706,-454.305022,-75299.247468
8,شپنا,32895084,ضشنا6009,1402-06-01,14000,call,0.003604,35.610024,-60.29616,9128532.572299
9,شپنا,32895084,طشنا6009,1402-06-01,14000,put,-0.003604,35.610024,3478.453487,-9128532.572299


In [84]:
from scipy.optimize import minimize

def calculate_exposure(n_options, row):
    print(row)
    # Calculate the exposure to each greek for selling n_options options
    delta_exposure = abs(row['holding'] - n_options[0] * row['delta'])
    vega_exposure = abs(n_options[0] * row['vega'])
    theta_exposure = abs(n_options[0] * row['theta'])
    return delta_exposure + vega_exposure + theta_exposure

def calculate_options_to_sell(row):
    print(row)
    # Initial guess for the number of options to sell
    n_options_initial = row['delta_neutral']
    # Bounds to ensure we sell a non-negative number of options
    bounds = [(0, None)]
    result = minimize(calculate_exposure, n_options_initial, args=(row,), bounds=bounds)
    return result.x[0]

In [85]:
fund_portfolio_options['startegy'] = fund_portfolio_options.apply(calculate_options_to_sell, axis=1)

stock_symbol            خودرو
holding              38372289
option_symbol        ضخود6015
maturity_date      1402-06-08
strike                   3000
type                     call
delta                0.774138
vega               383.160371
theta             -1020.14514
delta_neutral    49567.754081
Name: 0, dtype: object
[49567.75408094]
[49567.75408094]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices